In [14]:
"""
nltk processing: tokenizer, stemmer, lemmatizer and case conversion(lower_case) 
and also extract the ending punctions which can be used as an feature. 
"""
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import nltk
import gensim
import string
%matplotlib inline

pal = sns.color_palette()

In [10]:
#df_train, df_test = train_test_split(df, test_size=0.2, random_state=4)
df_train = pd.read_csv("train.csv",index_col=0)
df_test = pd.read_csv("test.csv",index_col=0)

In [11]:
df_train.shape, df_test.shape

((323432, 6), (80858, 6))

In [20]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
145367,145367,229850,229851,Is there only one consciousness?,Is consciousness one for everyone?,1
123169,123169,98309,199206,Which is a suitable solar panel installation p...,Which is a suitable solar panel installation p...,0
57880,57880,101653,101654,Is there any idea for a new and useful mini pr...,Are there any new topics in computer science t...,0
213043,213043,318261,318262,What does it mean if a girl cries after hurtin...,What does it mean if a girl cries without any ...,0
204901,204901,307914,307915,What are some wholesale drop shippers that hav...,What portrait lenses are compatible with the C...,0


In [36]:
porter = nltk.PorterStemmer()
wnl = nltk.WordNetLemmatizer()
def normalize_sentence(s):
    """
    normlize sentence, 
    @param s: a sentence 
    TODO: need to remove punctuation? 
    return List[word], with words stemmed,lemmatized, all lower case, 
    """
    try: 
        tokens = nltk.word_tokenize(s) #stem change why->whi 

        #stems = [porter.stem(t) for t in tokens] #stem change why->whi 
        words = [wnl.lemmatize(t).lower() for t in tokens]
    except:
        words = []
    return words

In [29]:
s="Why is FaceTime banned in UAE?"
r=normalize_sentence(s)
r

['why', 'is', 'facetime', 'banned', 'in', 'uae', '?']

In [32]:
df_train["question1"][:10]

145367                     Is there only one consciousness?
123169    Which is a suitable solar panel installation p...
57880     Is there any idea for a new and useful mini pr...
213043    What does it mean if a girl cries after hurtin...
204901    What are some wholesale drop shippers that hav...
5381      How can plan a trip for 3 and half days to Goa...
295129    Is our PM Modi doing the correct thing with 50...
33730     If gay marriage is legal, then what's to stop ...
105706    What are the day-to-day duties of a sales plan...
324456      Why doesn't it matter what people think of you?
Name: question1, dtype: object

In [35]:
tmp = df_train["question1"][:10].apply(lambda s: normalize_sentence(s))
tmp

145367             [is, there, only, one, consciousness, ?]
123169    [which, is, a, suitable, solar, panel, install...
57880     [is, there, any, idea, for, a, new, and, usefu...
213043    [what, doe, it, mean, if, a, girl, cry, after,...
204901    [what, are, some, wholesale, drop, shipper, th...
5381      [how, can, plan, a, trip, for, 3, and, half, d...
295129    [is, our, pm, modi, doing, the, correct, thing...
33730     [if, gay, marriage, is, legal, ,, then, what, ...
105706    [what, are, the, day-to-day, duty, of, a, sale...
324456    [why, doe, n't, it, matter, what, people, thin...
Name: question1, dtype: object

In [41]:
df_train["words_1"] = df_train["question1"].apply(lambda s: normalize_sentence(s))
df_train["words_2"] = df_train["question2"].apply(lambda s: normalize_sentence(s))
df_test["words_1"] = df_test["question1"].apply(lambda s: normalize_sentence(s))
df_test["words_2"] = df_test["question2"].apply(lambda s: normalize_sentence(s))

In [56]:
#extract ending punctuation, 'x' if no punctuation
def end_punc(s):
    if s:
        t = s[-1]
        return t if t in string.punctuation else 'x'
    else:
        return 'x'

In [57]:
df_train["end_punc_1"] = df_train["words_1"].apply(lambda s: end_punc(s))
df_train["end_punc_2"] = df_train["words_2"].apply(lambda s: end_punc(s))
df_test["end_punc_1"] = df_test["words_1"].apply(lambda s: end_punc(s))
df_test["end_punc_2"] = df_test["words_2"].apply(lambda s: end_punc(s))

In [58]:
df_train['end_punc_1'].value_counts()

?    320073
.      1417
x      1250
)       510
!        99
]        38
'        14
*         8
,         6
:         5
[         3
}         2
-         2
>         1
;         1
\         1
%         1
(         1
Name: end_punc_1, dtype: int64

In [62]:
#df_train["is_same_end_punc"] = (df_train['end_punc_1'] == df_train['end_punc_2'])
df_test["is_same_end_punc"] = (df_test['end_punc_1'] == df_test['end_punc_2'])

In [65]:
df_train['is_same_end_punc'].value_counts()

True     317064
False      6368
Name: is_same_end_punc, dtype: int64

In [67]:
df_test['is_same_end_punc'].value_counts()

True     79258
False     1600
Name: is_same_end_punc, dtype: int64

In [68]:
#save 
df_train.to_pickle("train_pp.pkl")
df_test.to_pickle("test_pp.pkl")